<a href="https://colab.research.google.com/github/GabKreuz/Maketing-Data-Analisys-and-Prediction/blob/main/Linear_Regression_Model_for_Marketing_Data_Analisys_and_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression Model for Marketing Data Analysis and Prediction

## **Objective**:

1. Analyze and comprehend the key factors and variables influencing lead generation;
2. Develop a prediction model to estimate sales returns and revenue based on marketing investment.

## Libraries and Prophylaxis

In [218]:
# Drive Mount
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [219]:
# Libraries and Resources Import
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [220]:
# Data Frame Import
df = pd.read_csv("/content/drive/MyDrive/Formação em Dados/Desafio 4/MKT.csv")

## Descriptive Analysis
The data is composed of 4 columns:
1. youtube: investment in marketing on youtube per month
2. facebook: investiment in marketing on facebook per month
3. newspaper: investment in marketing on newspaper per month
4. sales: sales revenue per month

The database has 171 rows, and all the data are of float type. No data preparation techniques were needed.

No information regarding the units and scale of the variables was provided. Therefore, considering that the database represents monthly values of a Brazilian company, for analysis purposes, it will be assumed that the unit of measurement is "Rs" for the investments collumns and "parts per Rs 1.000,00" for the sales column.

The main objective is to analyze and predict sales numbers based on marketing investments. To pursue this objective, the columns "youtube," "facebook," and "newspaper" will be considered as categorical and predictive variables. The "sales" column will be the target variable.

In [221]:
df.head(10)

,youtube,facebook,newspaper,sales
0,84.72,19.20,48.96,12.60
1,351.48,33.96,51.84,25.68
2,135.48,20.88,46.32,14.28
3,116.64,1.80,36.00,11.52
4,318.72,24.00,0.36,20.88
5,114.84,1.68,8.88,11.40
6,348.84,4.92,10.20,15.36
7,320.28,52.56,6.00,30.48
8,89.64,59.28,54.84,17.64
9,51.72,32.04,42.12,12.12


In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   youtube    171 non-null    float64
 1   facebook   171 non-null    float64
 2   newspaper  171 non-null    float64
 3   sales      171 non-null    float64
dtypes: float64(4)
memory usage: 5.5 KB


In [223]:
df.describe()

,youtube,facebook,newspaper,sales
count,171.000000,171.000000,171.000000,171.000000
mean,178.021053,27.671579,35.240000,16.922807
std,102.449597,17.913532,24.902918,6.314608
min,0.840000,0.000000,0.360000,1.920000
25%,91.080000,11.700000,13.740000,12.540000
50%,179.760000,26.760000,31.080000,15.480000
75%,262.980000,43.680000,50.880000,20.820000
max,355.680000,59.520000,121.080000,32.400000


##Exploratory Analysis

To conduct this analysis, a new exploration dataframe was created, to which two new attributes were added:

1. social_media: Marketing investments in social media (facebook + youTube) per month;
2. total: Total marketing investment per month.

Subsequently, a scatter plot was created by crossing all the data. It is observed that the original attributes (youTube, facebook, and newspaper) do not show any significant patterns or correlations. On the other hand, the attributes demonstrate noticeable correlations with the target (sales), all exhibiting a positive correlation.

The trendline tool, using the "ols" (ordinary least squares) method from the plotly.express library, was employed to interpret the relationship between the attributes and the target. From this analysis, the following conclusions were drawn:

1. The "newspaper" attribute shows a weak relationship with the target, with a coefficient of determination (r²) of 0.065;
2. "social_media" reveals a high relationship with the target, with an r² of 0.742;
3. The "youtube" attribute is the variable with the highest correlation with the target, presenting an r² of 0.612.

In [224]:
# Formating all the data to the same base (Rs)
df["sales"] = df["sales"].apply(lambda x: x*1000)

# Creation of a exploration dataframe
df_exp = df.sort_values(ascending=True,by="sales").reset_index()

# Total marketing investment
df_exp["total"] = df_exp[["youtube","facebook","newspaper"]].sum(axis=1)
# Marketing investment on social medias
df_exp["social_media"] = df_exp[["youtube","facebook"]].sum(axis=1)

df_exp = df_exp[["youtube","facebook","newspaper","social_media","total","sales"]]
df_exp.head()


,youtube,facebook,newspaper,social_media,total,sales
0,0.84,47.52,10.44,48.36,58.80,1920.0
1,4.92,13.92,6.84,18.84,25.68,3840.0
2,6.48,35.88,11.28,42.36,53.64,6360.0
3,15.72,0.48,30.72,16.20,46.92,6360.0
4,8.76,33.72,49.68,42.48,92.16,6600.0


In [225]:
px.scatter_matrix(df_exp)

In [226]:
px.scatter(df,x="youtube",y="sales",trendline="ols")

In [227]:
px.scatter(df_exp,x="newspaper",y="sales",trendline="ols")

In [228]:
px.scatter(df_exp,x="facebook",y="sales",trendline="ols")

In [229]:
px.scatter(df_exp,x="total",y="sales",trendline="ols")

In [230]:
px.scatter(df_exp,x="social_media",y="sales",trendline="ols")

## Modeling

In the model construction phase, the implementation of a simple linear regression model was demanded.

### Modeling Technique

* Linear Regression - LinearRegression - SKLEARN
[SKLEARN LinearRegression Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)
* Hyperparameter Optimization - GridSearchCV - SKLEARN
[SKLEARN GridSearchCV Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)

### Training and Testing Design

We divided the dataset into training and testing sets, using an 80/20% split and introducing a randomization component through the SKLEARN method.

To evaluate the model, we employed the metrics MSE (Mean Squared Error) and RMSE (Root Mean Squared Error) to penalize significant prediction errors.

[SKLEARN mean_squared_error Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error)

### Model Evaluation

The model yielded an RMSE of R$ 1781.42, representing an error of 10.1134% from the mean. Given the limited amount of available data, we deemed this result acceptable.

Due to the model's straightforward simplicity and transparent parameter approach, hyperparameter optimization did not have a noticeable impact on the outcomes. Therefore, we chose the non-optimized model as it delivers "the same result" with lower computational effort and less complex operations.

In [231]:
# Investment Characteristics (Attributes)
X = df[["youtube","facebook","newspaper"]]

# Sales Value (Target)
y = df[["sales"]]

# Train/Test Data Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=22)

In [232]:
# Model Training
regLinear = LinearRegression().fit(X_train,y_train)

In [233]:
# Model Testing
# Test Mass Prediction
yPred = regLinear.predict(X_test)

# Mean Squared Error
mse = mean_squared_error(y_test,yPred)
# Root Mean Squared Error
rmse = np.sqrt(mse)

print("mse =",mse)
print("rmse =",rmse,":",(rmse/yPred.mean())*100,"%")

mse = 3173443.73339519
rmse = 1781.416215654048 : 10.113414900518672 %


In [234]:
# Hyperparameter Optimization
parameters = {
    "copy_X" : [False,True],
    "fit_intercept" : [False,True],
    "positive" : [False,True]
}

regLinearGrid = GridSearchCV(LinearRegression(),parameters,refit="mean_squared_error",verbose=True)
regLinearOtm = regLinearGrid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [235]:
# Optimized Model Testing

yPredOtm = regLinearOtm.predict(X_test)
mseOtm = mean_squared_error(y_test,yPredOtm)
rmseOtm = np.sqrt(mseOtm)

print("mse =",mseOtm)
print("rmse =",rmseOtm,":",(rmseOtm/yPredOtm.mean())*100,"%")

mse = 3173443.733395189
rmse = 1781.4162156540478 : 10.113414900518666 %


In [236]:
# Comparison of Methods
improv = (rmseOtm/rmse - 1)*100
print("Improvement :",improv,"%")

Improvement : -1.1102230246251565e-14 %


## Evaluation

In the prediction and results analysis phase, the coefficient of determination of the model were examined, along with an assessment of the investment parameters. In other words, we sought to understand the impact of investments in marketing, on YouTube, Facebook, and newspapers, on the number of sales.

To conduct this analysis, it is crucial to bear in mind that the model's objective is to optimize marketing expenditures, i.e., maximize the number of sales. To achieve this goal, we adopted the following assumptions:

1. The total marketing investment was arbitrarily set at R$ 300.00.
2. The invested amount will be evenly distributed among the relevant parameters of this phase.

To perform this analysis, we will employ a truth table, exploring all possible combinations within the mentioned constraints.


###Results Evaluation

Initially, the model presented a coefficient of determination (R²) of 92.47%, indicating an extremely satisfactory fit to the utilized data and excellent predictive capability within the sample space.

Therefore, we can assume that the results presented in the analysis of parameters have a high probability of representing analytically useful data. Thus, the following conclusions were drawn from the results and model development:

1. When comparing the results obtained without marketing investment to the investment in each parameter individually, it is noted that Facebook is the marketing platform with the highest sales-generating capacity. At the same time, it is observed that investment in newspapers is extremely inefficient.

2. Despite the model's excellent fit to the data, as evidenced in the modeling section, it exhibits a high error. This suggests the need for improved model training, possibly with a larger database or the exploration of other modeling techniques.

###Next Steps

Based on the results, it would be beneficial to conduct a detailed analysis of the YouTube and newspapers platforms.

1. **YouTube:** It would be necessary to examine whether the target audience for marketing on the platform aligns with the company's consumer base.
2. **Newspapers:** Due to the very low performance and limited correlation with sales, it would be interesting to explore reallocating the investment to other marketing platforms.



In [237]:
# Coefficient of Determination

r2= regLinear.score(X_test,y_test)

print("r² =",r2)

r² = 0.9246994272537105


In [238]:
# Parameter Weight Analysis using a truth table
X_Analisys = pd.DataFrame()
X_Analisys["youtube"] = [100,150,150,300,0,0,0,0]
X_Analisys["facebook"] = [100,150,0,0,150,300,0,0]
X_Analisys["newspaper"] = [100,0,150,0,150,0,300,0]

y_analisys = regLinear.predict(X_Analisys)

Analisys_Results = X_Analisys[["youtube","facebook","newspaper"]]
Analisys_Results["Sales Prediction"] = y_analisys
Analisys_Results.head(8)

,youtube,facebook,newspaper,Sales Prediction
0,100,100,100,27871.930516
1,150,150,0,39251.144975
2,150,0,150,10929.058198
3,300,0,0,16744.614797
4,0,150,150,33435.588376
5,0,300,0,61757.675152
6,0,0,300,5113.501599
7,0,0,0,3546.016761
